# Terminology project 

## **Spacy trainable pipe**

### AGUIAR Mathilde NIAOURI Dimitra

#### M2 NLP

# Goal and tools 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Imports and packages 

In [2]:
!pip3 install 'spacy==3.4.3'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#!python -m spacy download en_core_web_sm
!pip install spacy-transformers

## Uncomment if you want to download one of the following model
# !python -m spacy download en_core_web_md
!python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 1.1 MB 56.6 MB/s 
     |████████████████████████████████| 4.7 MB 59.5 MB/s 
     |████████████████████████████████| 182 kB 73.4 MB/s 
     |████████████████████████████████| 6.6 MB 52.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 17 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
import spacy
import fnmatch
import os
import re
from os import path, walk
import shutil
import glob
import pandas as pd
from spacy.scorer import Scorer
from spacy.training import Example
from spacy.tokens import Doc

Downloading the curated dataset 

In [5]:
!wget https://raw.githubusercontent.com/MathildeAguiar/terminology_project/tree/main/data

# Data Preprocessing

Helpers to find files and create folders

In [6]:
def create_folder(path):
    """Create a folder if it doesn't already exist"""
    if not os.path.isdir(path):
        os.makedirs(path)
    return


def find_files(directory, pattern='*.final'):
    """Recursively finds all files matching the pattern."""
    files = []
    for root, dirnames, filenames in walk(directory):
        for filename in fnmatch.filter(filenames, pattern):
            files.append(path.join(root, filename))
    # sort the list, to avoid mismatch in the output files
    files = sorted(files)
    return files


Adapt the format of the IOB tags to be supported by Spacy. Spacy expects tags in the following format: 
```
B-ATTRIBUTE
```
So we convert the `B` into `B-NLP` and the `I` to `I-NLP`. 

In [7]:
def mapper(token):
  """
  Adapts the tags into a Spacy readable format
  """
  if token == "B":
    return "B-NLP"
  elif token == "I":
    return "I-NLP"
  else: 
    return token

In [8]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/training_data/iob_big 
!mkdir /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big

In [9]:
def retag_files(): 
    """ 
    Retag the original .final files by using the Spacy tag format.
    Save the result into a .iob file.    
    """
    # First process the training files
    directory_tg_train = "/content/drive/MyDrive/Terminology_project/curated/training_data/" 
    all_training_files = find_files(directory_tg_train) 
    for f in all_training_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_train+"iob_big/"+tmp[0].split(".")[0]+".iob"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue  
                new_file.write(new_txt)
            new_file.close()

    # 2nd process the dev files
    directory_tg_dev = "/content/drive/MyDrive/Terminology_project/curated/dev_data/"  
    all_dev_files = find_files(directory_tg_dev)
    for f in all_dev_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_dev+"iob_big/"+tmp[0].split(".")[0]+".iob"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue  
                new_file.write(new_txt)
            new_file.close()


In [10]:
retag_files()

### Convert the IOB files into *.spacy* binary files

In [11]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/training_data/iob_big/output 
!mkdir /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big/output 

In [12]:
## For the training set ##
# Uncomment the line you want in function of the model you are using 
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_sm /content/drive/MyDrive/Terminology_project/curated/training_data/iob /content/drive/MyDrive/Terminology_project/curated/training_data/iob/output

#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_md /curated/training_data/iob /curated/training_data/iob/output
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_lg /content/drive/MyDrive/Terminology_project/curated/training_data/iob_big /content/drive/MyDrive/Terminology_project/curated/training_data/iob_big/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /curated/training_data/iob /curated/training_data/iob/output

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob_big/output/2009-35-1-29-46.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob_big/output/2009-35-1-3-28.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob_big/output/2009-35-1-47-59.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_projec

In [13]:
## For the dev set ##
# Uncomment the line you want in function of the model you are using 
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_sm /content/drive/MyDrive/Terminology_project/curated/dev_data/iob /content/drive/MyDrive/Terminology_project/curated/dev_data/iob/output

#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_md /curated/dev_data/iob /curated/dev_data/iob/output
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_lg /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /curated/dev_data/iob /curated/dev_data/iob/output

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big/2009-35-4-597-635.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big/2010-36-1-1-30.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big/2011-37-3-421-454.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big/2012-

# Training the pipeline

First convert the base-config into a config file and then simply using the command line `spacy train`

In [44]:
# If the GPU is available run this cell 
!python -m spacy init fill-config /content/drive/MyDrive/Terminology_project/base_config_ner_only.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [46]:
!mkdir /content/drive/MyDrive/Terminology_project/output_clean_big  

mkdir: cannot create directory ‘/content/drive/MyDrive/Terminology_project/output_clean_big’: File exists


In [58]:
!python -m spacy train /content/config.cfg --output /content/drive/MyDrive/Terminology_project/output_clean_big --paths.train /content/drive/MyDrive/Terminology_project/curated/training_data/iob_big/output --paths.dev /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_big

ℹ Saving to output directory:
/content/drive/MyDrive/Terminology_project/output_clean_big
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-12-06 10:20:16,782] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-06 10:20:16,792] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-12-06 10:20:16,796] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-06 10:20:18,463] [INFO] Added vectors: en_core_web_lg
INFO:spacy:Added vectors: en_core_web_lg
tcmalloc: large alloc 1233977344 bytes == 0x59f08000 @  0x7f4f2389b2a4 0x7f4f181dbe09 0x7f4f181dacdf 0x7f4f181d7675 0x7f4f181d7e2e 0x4f7585 0x49ca7c 0x55e571 0x5d7cf1 0x49ced5 0x7f4de3792d78 0x7f4de379570e 0x7f4de379d5a7 0x7f4de37a0e85 0x5d813c 0x55f3fd 0x5d7c18 0x49ec69 0x7f4de3792d78 0x7f4de379570e 0x7f4de379ce4b 0x5d746e 0x5d813c 0x560200 0x55e858 0x5d7cf1 0x49ced5 0x4fe99

You should obtain 2 models: `model-best` and `model-base`. To test the trained Pipeline it's better to use `model-best`. 

# Testing on the test dataset

### Preprocessinf the test files 

In [48]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/test_data/iob_big

mkdir: cannot create directory ‘/content/drive/MyDrive/Terminology_project/curated/test_data/iob_big’: File exists


In [49]:
def retag_test_files():  
    """ 
    Retag the original .final files by using the Spacy tag format.
    Save the result into a .iob file.    
    """
    # First process the training files
    directory_tg_test = "/content/drive/MyDrive/Terminology_project/curated/test_data/" 
    all_test_files = glob.glob(directory_tg_test + "/*.final")
    for f in all_test_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_test+"iob_big/"+tmp[0].split(".")[0]+".iob"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue 
                new_file.write(new_txt)
            new_file.close()

In [50]:
retag_test_files()

In [51]:
path = r'/content/drive/MyDrive/Terminology_project/curated/test_data/iob_big'
all_files = glob.glob(path + "/*.iob")

l = []
cols = ['Token', 'Tag']

for filename in all_files:
    df = pd.read_csv(filename, names = cols,on_bad_lines='skip', sep='\s+', engine='python')
    l.append(df)

# Our tokens from test set are stored in a Dataframe
test_df = pd.concat(l, axis=0, ignore_index=True)

# drop nan values
test_df = test_df.dropna()
test_df['Token'].fillna('', inplace=True)


## Using Scorer to test on the test sentences 

In [52]:
test_df.head()
test_df.iloc[88]
print(len(test_df['Token']))
print(len(test_df['Tag']))

8316
8316


### Using the command line

In [55]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/test_data/iob_big/output

mkdir: cannot create directory ‘/content/drive/MyDrive/Terminology_project/curated/test_data/iob_big/output’: File exists


In [63]:
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_lg /content/drive/MyDrive/Terminology_project/curated/test_data/iob_big /content/drive/MyDrive/Terminology_project/curated/test_data/iob_big/output


ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/iob_big/output/2009-35-4-559-595.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/iob_big/output/2010-36-1-31-69.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/iob_big/output/2011-37-3-455-488.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_lg'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curat

In [60]:
!python -m spacy evaluate /content/drive/MyDrive/Terminology_project/output_clean_big/model-best /content/drive/MyDrive/Terminology_project/curated/test_data/iob_big/output

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     -    
NER P   65.12
NER R   61.95
NER F   63.49
SPEED   4501 


=============================== NER (per type) ===============================

          P       R       F
NLP   65.12   61.95   63.49

